무신사 크롤링 과제 (아직 하는중)

In [1]:
# pip install selenium==4.8.2

In [2]:
# pip install webdriver_manager

In [3]:
from bs4 import BeautifulSoup
import selenium
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
import numpy as np
import time
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import re
print(selenium.__version__)

4.8.2


In [4]:
options = webdriver.ChromeOptions()
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

In [5]:
codimap_list = []
for page in range(1): 
    url = 'https://www.musinsa.com/app/codimap/lists?style_type=&tag_no=&brand=&display_cnt=60&list_kind=big&sort=comment_cnt&page={}'.format(page)
    driver.get(url)
    try:
        driver.maximize_window()
    except:
        pass

    columns = ['codimap_category', 'codimap_title', 'codimap_date', 'views', 'comment_numbers', 'codimap_explain', 'codimap_hashtag', 'codimap_imgurl']
    values = []
    soup = BeautifulSoup(driver.page_source, 'lxml')

    data_rows = soup.find_all('li', attrs={'class':'style-list-item'})

    for i, row in enumerate(data_rows):
        try:
            interval = 1 

            
            prev_height = driver.execute_script("return document.body.scrollHeight")

            
            while True:
                
                driver.execute_script("window.scrollTo(0, document.body.scrollHeight)")

                
                time.sleep(interval)

                
                curr_height = driver.execute_script("return document.body.scrollHeight")
                if curr_height == prev_height:
                    break

                prev_height = curr_height

            try:
                elem = WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.XPATH, '/html/body/div[3]/div[3]/form/div[4]/div/ul/li[60]/div[1]')))
                 
                if elem:
                    print("스크롤 완료")
            except:
                print("스크롤 실패")
                pass

            print('{}페이지 {}번째 코디맵 크롤링'.format(page, i+1))
            blank = []

            codimap_category = row.find('span', attrs={'class':'style-list-information__text'})
            if codimap_category: 
                codimap_category = codimap_category.get_text().strip()
                blank.append(codimap_category)
            else:
                blank.append('something is wrong')
                print('{}페이지 {}번째 코디맵에서 codimap_category 가져올때 문제발생'.format(page, i+1))
                continue

            codimap_title = row.find('strong', attrs={'class':'style-list-information__title'})
            if codimap_title:  
                codimap_title = codimap_title.get_text().strip()
                blank.append(codimap_title)
            else:
                blank.append('something is wrong')
                print('{}페이지 {}번째 코디맵에서 codimap_title 가져올때 문제발생'.format(page, i+1))
                continue
                
            codimap_date = row.find_all('span', attrs={'class':'post-information__text'})
            if codimap_date:  
                codimap_date = codimap_date[0].get_text().strip()
                blank.append(codimap_date)
            else:
                blank.append('something is wrong')
                print('{}페이지 {}번째 코디맵에서 codimap_date 가져올때 문제발생'.format(page, i+1))
                continue

                
            views = row.find_all('span', attrs={'class':'post-information__text'})
            if views: 
                views = views[1].get_text().strip().replace('조회 ', '')
                blank.append(views)
            else:
                blank.append('something is wrong')
                print('{}페이지 {}번째 코디맵에서 views 가져올때 문제발생'.format(page, i+1))
                continue

                
            comment_numbers = row.find_all('span', attrs={'class':'post-information__text'})
            if comment_numbers: 
                comment_numbers = comment_numbers[2].get_text().strip().replace('댓글 ', '')
                blank.append(comment_numbers)

            else:
                blank.append('something is wrong')
                print('{}페이지 {}번째 코디맵에서 codimap_date 가져올때 문제발생'.format(page, i+1))
                continue


            codi_element_xpath = driver.find_element(By.XPATH, '/html/body/div[3]/div[3]/form/div[4]/div/ul/li[{}]/div[1]/a/div/img'.format(i+1))
            codi_element_xpath.click()  
            time.sleep(2)

            soup = BeautifulSoup(driver.page_source, 'lxml')
            codimap_explain = soup.find('p', attrs={'class':'styling_txt'})
            if codimap_explain: 
                codimap_explain = codimap_explain.get_text().strip()
                if '\r' in codimap_explain:
                    codimap_explain = codimap_explain.replace('\r', ' ')
                blank.append(codimap_explain)
            else:
                blank.append('something is wrong')
                print('{}페이지 {}번째 코디맵에서 codimap_explain 가져올때 문제발생'.format(page, i+1))
                continue

            codimap_hashtags = soup.find_all('a', attrs={'class':'ui-tag-list__item'})
            if codimap_hashtags: 
                codimap_hastag_list = []
                for codimap_hashtag in codimap_hashtags:
                    codimap_hastag_list.append(codimap_hashtag.get_text().strip())
                hashtags = ','.join(codimap_hastag_list)
                blank.append(hashtags)
            else:
                blank.append('something is wrong')
                print('{}페이지 {}번째 코디맵에서 codimap_hashtags 가져올때 문제발생'.format(page, i+1))
                continue
                
            codimap_imgurl = soup.find('img', attrs={'class':'photo'})
            if codimap_imgurl: 
                image_url = codimap_imgurl['src']
                if image_url.startswith('//'):
                    image_url = 'https:' + image_url
                blank.append(image_url)
            else:
                blank.append('something is wrong')
                print('{}페이지 {}번째 코디맵에서 codimap_imgurl 가져올때 문제발생'.format(page, i+1))
                continue

            
            values.append(blank)
            driver.back()
            driver.implicitly_wait(2)

        except:
            print('{}번째 코디맵에서 문제발생'.format(i+1))
            pass

    df = pd.DataFrame(values, columns = columns)
    codimap_list.append(df)
    

스크롤 완료
0페이지 1번째 코디맵 크롤링
스크롤 완료
0페이지 2번째 코디맵 크롤링
스크롤 완료
0페이지 3번째 코디맵 크롤링
스크롤 완료
0페이지 4번째 코디맵 크롤링
스크롤 완료
0페이지 5번째 코디맵 크롤링
스크롤 완료
0페이지 6번째 코디맵 크롤링
스크롤 완료
0페이지 7번째 코디맵 크롤링
스크롤 완료
0페이지 8번째 코디맵 크롤링
스크롤 완료
0페이지 9번째 코디맵 크롤링
스크롤 완료
0페이지 10번째 코디맵 크롤링
스크롤 완료
0페이지 11번째 코디맵 크롤링
스크롤 완료
0페이지 12번째 코디맵 크롤링
스크롤 완료
0페이지 13번째 코디맵 크롤링
스크롤 완료
0페이지 14번째 코디맵 크롤링
스크롤 완료
0페이지 15번째 코디맵 크롤링
스크롤 완료
0페이지 16번째 코디맵 크롤링
스크롤 완료
0페이지 17번째 코디맵 크롤링
스크롤 완료
0페이지 18번째 코디맵 크롤링
스크롤 완료
0페이지 19번째 코디맵 크롤링
스크롤 완료
0페이지 20번째 코디맵 크롤링
스크롤 완료
0페이지 21번째 코디맵 크롤링
스크롤 완료
0페이지 22번째 코디맵 크롤링
스크롤 완료
0페이지 23번째 코디맵 크롤링
스크롤 완료
0페이지 24번째 코디맵 크롤링
스크롤 완료
0페이지 25번째 코디맵 크롤링
스크롤 완료
0페이지 26번째 코디맵 크롤링
스크롤 완료
0페이지 27번째 코디맵 크롤링
스크롤 완료
0페이지 28번째 코디맵 크롤링
스크롤 완료
0페이지 29번째 코디맵 크롤링
스크롤 완료
0페이지 30번째 코디맵 크롤링
스크롤 완료
0페이지 31번째 코디맵 크롤링
스크롤 완료
0페이지 32번째 코디맵 크롤링
스크롤 완료
0페이지 33번째 코디맵 크롤링
스크롤 완료
0페이지 34번째 코디맵 크롤링
스크롤 완료
0페이지 35번째 코디맵 크롤링
스크롤 완료
0페이지 36번째 코디맵 크롤링
스크롤 완료
0페이지 37번째 코디맵 크롤링
스크롤 완료
0페이지 38번째 코디맵 크롤링
스크롤 완료
0페이지 39번째 코디맵 크롤링
스크롤 완료
0페이지 40번째 코디맵 크롤링
스크롤 완료
0페

In [7]:
df_ = pd.concat(codimap_list).reset_index(drop=True) 
df_.head(10)

,codimap_category,codimap_title,codimap_date,views,comment_numbers,codimap_explain,codimap_hashtag,codimap_imgurl
0,캐주얼,톤온톤 코디,21.07.02,"16,096",49,편하게 입기 좋은 반팔 티셔츠와 와이드 팬츠를 연출하고 크로스 백으로 마무리한 캐주얼 룩,"#톤온톤,#뉴트럴톤,#캠퍼스,#피크닉,#와이드핏,#오버핏,#꾸안꾸,#여름,#캐주얼,...",https://image.msscdn.net/images/codimap/detail...
1,캐주얼,블랙 매니아,21.02.23,"34,684",46,편안한 무드의 조거 팬츠에 블랙 스웨트셔츠를 매치한 후 볼캡으로 마무리한 캐주얼 룩,"#봄,#캐주얼,#라퍼지스토어,#다이아몬드 레이라,#나이키,#엠프렌즈,#모티패스트,#...",https://image.msscdn.net/images/codimap/detail...
2,캐주얼,오늘의 코디,22.03.22,"12,208",38,로고 디테일이 돋보이는 스웨트셔츠와 데님 팬츠를 코디하고 스니커즈로 완성한 캐주얼 룩,"#로고플레이,#오버핏,#스티치,#캠퍼스,#봄,#캐주얼,#유니폼브릿지,#아디다스,#4...",https://image.msscdn.net/images/codimap/detail...
3,댄디,추천 댄디 룩,21.09.15,"20,193",27,담백한 디테일의 니트와 셔츠를 레이어드하고 데님 팬츠를 더해 완성한 댄디 룩,"#데이트,#그래픽,#레이어드룩,#가을,#댄디,#엘무드,#뮤지엄바이비컨,#테이크이지,...",https://image.msscdn.net/images/codimap/detail...
4,캐주얼,릴랙스 룩,21.02.23,"23,547",22,플라워 자수가 눈에 띄는 니트에 조거 팬츠를 더한 후 레드 볼캡으로 컬러 포인트를 ...,"#봄,#캐주얼,#카시오,#코닥,#제로,#와릿이즌,#뉴발란스,#커버낫,#디지털,#미니...",https://image.msscdn.net/images/codimap/detail...
5,스트릿,힙하다 힙해,21.02.03,"35,297",21,레더 블루종 점퍼에 오버사이즈 니트와 조거 팬츠를 매치하고 블루 컬러 볼캡으로 멋을...,"#봄,#스트릿,#제로,#월간,#라퍼지스토어,#키치오브제,#블론드나인,#47브랜드,#...",https://image.msscdn.net/images/codimap/detail...
6,캐주얼,심플한 코디,21.07.30,"39,596",18,로고 프린팅이 돋보이는 반팔 티셔츠와 스웨트 팬츠를 매치하고 스니커즈로 완성한 이지...,"#로고플레이,#편안한,#원마일웨어,#와이드핏,#여름,#캐주얼,#리,#세미콜론 아이웨...",https://image.msscdn.net/images/codimap/detail...
7,댄디,데이트 코디,21.12.24,"10,317",18,깔끔한 무드의 니트에 데님 팬츠를 스타일링하고 스니커즈로 마무리한 댄디 룩,"#남친룩,#레더,#데이트,#테이퍼드핏,#오버핏,#소개팅,#스티치,#겨울,#댄디,#쿠...",https://image.msscdn.net/images/codimap/detail...
8,댄디,훈훈 남친 룩,21.03.24,"19,352",16,체크 패턴이 돋보이는 블레이저에 셔츠와 데님 팬츠를 더하고 담백한 구두로 마무리한 ...,"#봄,#댄디,#모디파이드,#반,#락포트,#수아레,#슈트/블레이저 재킷,#기타 시계,...",https://image.msscdn.net/images/codimap/detail...
9,캐주얼,지금 입기 좋아,21.08.31,"20,223",16,오버 핏 스웨트셔츠와 베이지 컬러 팬츠를 코디하고 더비 슈즈로 완성한 캐주얼 룩,"#꾸안꾸,#남친룩,#핀턱,#캠퍼스,#와이드핏,#여름,#캐주얼,#브래디백,#와일드 브...",https://image.msscdn.net/images/codimap/detail...


In [8]:
df_.shape

(60, 8)

In [9]:
df_.to_csv('김현호_무신사_크롤링.csv', encoding = 'utf-8-sig') 

In [10]:
driver.close()